In [1]:

from pyLIMA.simulations import simulator
from pyLIMA import event
from pyLIMA import telescopes
from pyLIMA.models import PSBL_model
from IPython.core.debugger import Pdb 
from astropy.table import Table
import copy
import numpy as np

from pyLIMA.toolbox import time_series
RA = 267.92497054815516 
DEC = -29.152232510353276

tlsst = 60413.26382860778+2400000.5
tstart_Roman = tlsst + 3*365 #Roman is expected to be launch in may 2027


new_creation = event.Event(ra=RA, dec=DEC)
new_creation.name = 'An event observed by Roman'

magu = np.loadtxt('/home/anibal/roman_rubin/simulation/lightcurve_u.txt')
magg = np.loadtxt('/home/anibal/roman_rubin/simulation/lightcurve_g.txt')
magr = np.loadtxt('/home/anibal/roman_rubin/simulation/lightcurve_r.txt')
magi = np.loadtxt('/home/anibal/roman_rubin/simulation/lightcurve_i.txt')
magz = np.loadtxt('/home/anibal/roman_rubin/simulation/lightcurve_z.txt')
magy = np.loadtxt('/home/anibal/roman_rubin/simulation/lightcurve_y.txt')

magu[:,0] = magu[:,0]+2400000.5
magg[:,0] = magg[:,0]+2400000.5
magr[:,0] = magr[:,0]+2400000.5
magi[:,0] = magi[:,0]+2400000.5
magz[:,0] = magz[:,0]+2400000.5
magy[:,0] = magy[:,0]+2400000.5


Roman1 = simulator.simulate_a_telescope(name='W149', time_start=tstart_Roman,time_end=tstart_Roman+72,sampling=0.25, 
                                        location='Space',camera_filter='W149',uniform_sampling=True, astrometry=False)
Roman2 = simulator.simulate_a_telescope(name='W149',time_start =tstart_Roman+107+72,time_end=tstart_Roman+107+72*2,sampling=0.25, 
                                        location='Space',camera_filter='W149',uniform_sampling=True, astrometry=False)
Roman3 = simulator.simulate_a_telescope(name='W149',time_start=tstart_Roman+107+72*2+113, time_end=tstart_Roman+107+72*3+113,sampling=0.25, 
                                        location='Space',camera_filter='W149',uniform_sampling=True, astrometry=False)
Roman4 = simulator.simulate_a_telescope(name='W149',time_start=tstart_Roman+107+72*3+113+838.36, time_end=tstart_Roman+107+72*4+113+838.36,sampling=0.25, 
                                        location='Space',camera_filter='W149',uniform_sampling=True, astrometry=False)
Roman5 = simulator.simulate_a_telescope(name='W149',time_start=tstart_Roman+107+72*4+113*2+838.36,time_end=tstart_Roman+107+72*5+113*2+838.36,sampling=0.25, 
                                        location='Space',camera_filter='W149',uniform_sampling=True, astrometry=False)
Roman6 = simulator.simulate_a_telescope(name='W149',time_start=tstart_Roman+107+72*4+113*2+838.36+72+107, time_end=tstart_Roman+107+72*4+113*2+838.36+72+107+72,sampling=0.25, 
                                        location='Space',camera_filter='W149',uniform_sampling=True, astrometry=False)
Roman_tot = simulator.simulate_a_telescope(name='W149',time_start=tstart_Roman+107+72*5+113*2+838.36+107, time_end=tstart_Roman+107+72*5+113*2+838.36+107+72,sampling=0.25, 
                                        location='Space',camera_filter='W149',uniform_sampling=True, astrometry=False)

new_array = np.r_[Roman1.lightcurve_flux,Roman2.lightcurve_flux,Roman3.lightcurve_flux,Roman4.lightcurve_flux,Roman5.lightcurve_flux,Roman6.lightcurve_flux]
new_table = time_series.construct_time_series(new_array,['time','flux','err_flux'],['JD','W/m^2','W/m^2'])

Roman_tot.lightcurve_flux = new_table

ephemerides = np.loadtxt('/home/anibal/files_db/james_webb.txt')
ephemerides[:,0] = ephemerides[:,0]- 2400000.5+2400000.5
ephemerides[:,3] *=  60*300000/150000000
deltaT = tlsst-ephemerides[:,0][0]
ephemerides[:,0] = ephemerides[:,0]+deltaT

Roman_tot.location = 'Space'
Roman_tot.spacecraft_name = 'WFIRST_W149'
Roman_tot.spacecraft_positions = {'astrometry':[],'photometry': ephemerides}

new_creation.telescopes.append(Roman_tot)

LSST_u = telescopes.Telescope(name='u', camera_filter='u', location='Earth', light_curve=magu.astype(float),
                              light_curve_names = ['time','mag','err_mag'], light_curve_units = ['JD','mag','mag'])

LSST_g = telescopes.Telescope(name='g', camera_filter='g', location='Earth', light_curve=magg.astype(float),
                              light_curve_names = ['time','mag','err_mag'], light_curve_units = ['JD','mag','mag'])


LSST_r = telescopes.Telescope(name='r', camera_filter='r', location='Earth', light_curve=magr.astype(float),
                              light_curve_names = ['time','mag','err_mag'],light_curve_units = ['JD','mag','mag'])


LSST_i = telescopes.Telescope(name='i', camera_filter='i', location='Earth', light_curve=magi.astype(float),
                              light_curve_names = ['time','mag','err_mag'],light_curve_units = ['JD','mag','mag'])


LSST_z = telescopes.Telescope(name='z', camera_filter='z', location='Earth', light_curve=magz.astype(float),
                              light_curve_names = ['time','mag','err_mag'], light_curve_units = ['JD','mag','mag'])

LSST_y = telescopes.Telescope(name='y', camera_filter='y', location='Earth', light_curve=magy.astype(float),
                              light_curve_names = ['time','mag','err_mag'], light_curve_units = ['JD','mag','mag'])


new_creation.telescopes.append(LSST_u)
new_creation.telescopes.append(LSST_g)
new_creation.telescopes.append(LSST_r)
new_creation.telescopes.append(LSST_i)
new_creation.telescopes.append(LSST_z)
new_creation.telescopes.append(LSST_y)



t0 = 2462039.664977124
my_own_parameters = [2462039.664977124, 0.5108280943700578, 132.7713496832835, 0.6288009670087173, 0.8929565891217635, 161.33275600801886, 0.4913914105623395, -0.0743343256786935]

my_own_model = PSBL_model.PSBLmodel(new_creation ,parallax=['Full', t0])
pyLIMA_parameters_1 = my_own_model.compute_pyLIMA_parameters(my_own_parameters)

# if coment this line and run the next block code this error dissapear
simulator.simulate_lightcurve_flux(my_own_model, pyLIMA_parameters_1)



/home/anibal/anaconda3/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 41472 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/anibal/anaconda3/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 41472 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/anibal/anaconda3/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/anibal/anaconda3/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/anibal/anaconda3/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 25 of "dubious

Parallax(Full) estimated for the telescope W149: SUCCESS


/home/anibal/anaconda3/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utcut1" yielded 25 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/anibal/anaconda3/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 137 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/anibal/anaconda3/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 137 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/anibal/anaconda3/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utcut1" yielded 137 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/anibal/anaconda3/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 142 of "dubious

Parallax(Full) estimated for the telescope u: SUCCESS
Parallax(Full) estimated for the telescope g: SUCCESS
Parallax(Full) estimated for the telescope r: SUCCESS
Parallax(Full) estimated for the telescope i: SUCCESS
Parallax(Full) estimated for the telescope z: SUCCESS
Parallax(Full) estimated for the telescope y: SUCCESS


ValueError: lam < 0 or lam contains NaNs

In [ ]:
magstar = [25.151799999999998, 31.103, 28.095, 26.749, 25.85, 25.408, 25.207]


telescope_bands = ['F146', 'u', 'g', 'r','i','z','y']
cero_p = [27.615, 27.03, 28.38, 28.16, 27.85,  27.46, 26.68]
mag_sat = [14.8, 14.7, 15.7, 15.8, 15.8, 15.3, 13.9]
filterset = ['w146','u','g','r','i','z','y']

fs = []
my_own_flux_parameters = []
np.random.seed(51)
for m in range(len(magstar)):
    ZP = cero_p[m] 
    mag_baseline = magstar[m]
    flux_baseline = 10**((ZP-mag_baseline)/2.5)
    g = np.random.uniform(0,1)
    f_source = flux_baseline/(1+g)
    fs.append(f_source)
    my_own_flux_parameters.append(f_source)
    my_own_flux_parameters.append(g)
my_own_parameters += my_own_flux_parameters

pyLIMA_parameters = my_own_model.compute_pyLIMA_parameters(my_own_parameters)

simulator.simulate_lightcurve_flux(my_own_model, pyLIMA_parameters)